In [1]:
import nb_importHooks
import Plot

dcc_plot.dcc_plot()  #background map

con = psycopg2.connect(database=dbname, user=username)
cur = con.cursor()
df_roads = gpd.GeoDataFrame.from_file(gis_file_path+'lines.shp') #as WGS84 latlon
wgs84  = pyproj.Proj(init='epsg:4326')  #WGS84
bng    = pyproj.Proj(init='epsg:27700') #british national grid

dt_start  = datetime.datetime.strptime('2016-12-05_00:00:00' , "%Y-%m-%d_%H:%M:%S" )
dt_end    = datetime.datetime.strptime('2016-12-10_00:00:00' , "%Y-%m-%d_%H:%M:%S" )
sql = "SELECT ways.gid, SUM(linkcount.count), ways.the_geom FROM ways, linkcount  WHERE linkcount.gid::int=ways.gid AND linkcount.timestamp>'%s' AND linkcount.timestamp<'%s'  GROUP BY ways.gid;"%(dt_start,dt_end)
print sql 
df_links = gpd.GeoDataFrame.from_postgis(sql,con,geom_col='the_geom')
#    pdb.set_trace()  #GROUP BY , SUM 
for i in range(0,df_links.shape[0]): 
    link = df_links.iloc[i]
    lons = link['the_geom'].coords.xy[0] #coordinates in latlon
    lats = link['the_geom'].coords.xy[1]
    gid = int(link.gid) 
    xs=[]
    ys=[]
    n_segments = len(lons)
    for j in range(0, n_segments):
        (x,y) = pyproj.transform(wgs84, bng, lons[j], lats[j]) #project to BNG -- uses nonISO lonlat convention  #TODO faster to cache this! 
        xs.append(x)
        ys.append(y)
    color='y'
    lw = int(link['sum']/10000)
    plot(xs, ys, color, linewidth=lw)  

#show bluetooth sites
sql = "SELECT * from sensorsite;"
df_sites = pd.read_sql_query(sql, con)
plot(df_sites.easting,df_sites.northing, 'bo')

importing Jupyter notebook from Plot.ipynb
